In [2]:
import tensorflow as tf
import os

In [3]:
#### IF USE GOOGLE COLAB - upload kagle.json to drive
# from google.colab import files
# files.upload()
# pip install -q kaggle
# mkdir -p ~/.kaggle
# cp kaggle.json ~/.kaggle/
# !kaggle datasets download -d moltean/fruits #kaggle
# !unzip fruits.zip 

base_dir = './fruits-360/Training'

In [4]:
IMAGE_SIZE = 224
BATCH_SIZE  = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    horizontal_flip=True,
    validation_split=0.1,
    fill_mode='nearest' 
)

train_generator = datagen.flow_from_directory(
    base_dir, 
    target_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    subset='training'
)

Found 60955 images belonging to 131 classes.


In [11]:
print(train_generator.class_indices)

labels = '\n'.join(sorted(train_generator.class_indices.keys()))

with open('labels.txt','w') as f:
    f.write(labels)

{'Apple Braeburn': 0, 'Apple Crimson Snow': 1, 'Apple Golden 1': 2, 'Apple Golden 2': 3, 'Apple Golden 3': 4, 'Apple Granny Smith': 5, 'Apple Pink Lady': 6, 'Apple Red 1': 7, 'Apple Red 2': 8, 'Apple Red 3': 9, 'Apple Red Delicious': 10, 'Apple Red Yellow 1': 11, 'Apple Red Yellow 2': 12, 'Apricot': 13, 'Avocado': 14, 'Avocado ripe': 15, 'Banana': 16, 'Banana Lady Finger': 17, 'Banana Red': 18, 'Beetroot': 19, 'Blueberry': 20, 'Cactus fruit': 21, 'Cantaloupe 1': 22, 'Cantaloupe 2': 23, 'Carambula': 24, 'Cauliflower': 25, 'Cherry 1': 26, 'Cherry 2': 27, 'Cherry Rainier': 28, 'Cherry Wax Black': 29, 'Cherry Wax Red': 30, 'Cherry Wax Yellow': 31, 'Chestnut': 32, 'Clementine': 33, 'Cocos': 34, 'Corn': 35, 'Corn Husk': 36, 'Cucumber Ripe': 37, 'Cucumber Ripe 2': 38, 'Dates': 39, 'Eggplant': 40, 'Fig': 41, 'Ginger Root': 42, 'Granadilla': 43, 'Grape Blue': 44, 'Grape Pink': 45, 'Grape White': 46, 'Grape White 2': 47, 'Grape White 3': 48, 'Grape White 4': 49, 'Grapefruit Pink': 50, 'Grapefrui

In [12]:
IMG_SHAPE = (IMAGE_SIZE,IMAGE_SIZE,3)

base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights='imagenet'
)

In [13]:
base_model.trainable = False

In [14]:
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.Conv2D(32,3,activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(131, activation='softmax')
])

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(), 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [16]:
epochs = 10
history = model.fit(
    train_generator,
    epochs= epochs,
   steps_per_epoch=200
) 

Epoch 1/10
200/200 [==============================] - 694s 3s/step - loss: 3.7535 - accuracy: 0.1741
Epoch 2/10
200/200 [==============================] - 819s 4s/step - loss: 1.5945 - accuracy: 0.5621
Epoch 3/10
200/200 [==============================] - 550s 3s/step - loss: 1.0034 - accuracy: 0.6999
Epoch 4/10
200/200 [==============================] - 567s 3s/step - loss: 0.8028 - accuracy: 0.7568
Epoch 5/10
200/200 [==============================] - 452s 2s/step - loss: 0.6733 - accuracy: 0.7966
Epoch 6/10
200/200 [==============================] - 540s 3s/step - loss: 0.6070 - accuracy: 0.8074
Epoch 7/10
200/200 [==============================] - 428s 2s/step - loss: 0.5575 - accuracy: 0.8237
Epoch 8/10
200/200 [==============================] - 493s 2s/step - loss: 0.5231 - accuracy: 0.8348
Epoch 9/10
200/200 [==============================] - 416s 2s/step - loss: 0.4773 - accuracy: 0.8497
Epoch 10/10
200/200 [==============================] - 695s 3s/step - loss: 0.4659 - accura

In [ ]:
save_model_dir = ''
tf.saved_model.save(model,save_model_dir)
converter = tf.lite.TFLiteConverter.from_saved_model(save_model_dir)

tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
      f.write(tflite_model)

In [ ]:
from google.colab import files
files.download('model.tflite')
files.download('labels.txt')